In [ ]:
# Instalar las dependencias necesarias
!pip install requests markdownify

# Importar las bibliotecas necesarias
import requests
import json
from markdownify import markdownify as md
import time

# Configuración de las solicitudes
graphql_url = "https://yourdomain.com/graphql"
headers = {
    "Authorization": "Bearer YOUR API KEY",
    "Content-Type": "application/json"
}

# Primera solicitud GraphQL para obtener la lista de páginas
query_list = '''
query Request {
  pages {
    list(orderBy: PATH) {
      id
      path
      title
      contentType
    }
  }
}
'''

response = requests.post(graphql_url, headers=headers, json={"query": query_list})

# Verificar si la solicitud fue exitosa
if response.status_code != 200:
    print(f"Error en la solicitud inicial: {response.status_code}")
    print(response.text)
else:
    # Imprimir la respuesta completa para depuración
    print("Respuesta completa de la solicitud inicial:")
    print(response.text)

    pages_list = response.json().get("data", {}).get("pages", {}).get("list", [])

    # Debugging output
    print(f"Se encontraron {len(pages_list)} páginas.")

    # Iterar sobre cada página y procesar su contenido
    for page in pages_list:
        page_id = page["id"]

        # Segunda solicitud GraphQL para obtener los detalles de una página
        query_details = f'''
        query Request {{
          pages {{
            single (id: {page_id}) {{
              path
              title
              description
              content
              contentType
              createdAt
              updatedAt
            }}
          }}
        }}
        '''

        response = requests.post(graphql_url, headers=headers, json={"query": query_details})

        # Verificar si la solicitud fue exitosa
        if response.status_code != 200:
            print(f"Error al obtener detalles de la página ID {page_id}: {response.status_code}")
            print(response.text)
            continue

        # Extraer los detalles de la página
        page_details = response.json().get("data", {}).get("pages", {}).get("single", {})

        if not page_details:
            print(f"Error: No se encontraron detalles para la página ID {page_id}")
            continue

        # Configurar las variables para el contenido de la página
        title = page_details.get("title", "Sin título")
        html_content = page_details.get("content", "")
        content_type = page_details.get("contentType", "text")

        # Imprimir el tipo de contenido para depuración
        print(f"Página ID {page_id}: contentType es {content_type}")

        # Convertir el contenido de HTML a Markdown si es necesario
        if content_type == "html":
            markdown_content = md(html_content)
        else:
            markdown_content = html_content

        # Tercera solicitud para crear un documento en Outline
        outline_url = "https://app.getoutline.com/api/documents.create"
        outline_headers = {
            "authorization": "Bearer YOUR API KEY",
            "Content-Type": "application/json"
        }
        data = {
            "title": title,
            "text": markdown_content,
            "collectionId": "YOUR COLLECTION ID",
            "template": False,
            "publish": False
        }

        response = requests.post(outline_url, headers=outline_headers, data=json.dumps(data))

        # Comprobar la respuesta de la creación del documento
        if response.status_code == 200:
            print(f"Documento '{title}' creado exitosamente.")
        else:
            print(f"Error al crear el documento '{title}': {response.status_code}")
            print(response.text)

        # Pausa entre solicitudes
        time.sleep(1)